# Скрипт для загрузки данных из EXCEL в SQL (PostgreSQL)

In [1]:
# импортируем pandas
import pandas as pd

In [2]:
# загружаем наш файл Excel или CSV (для excel используйте read_excel)
# обращайте внимание на тип разделителя значений (в случае с excel - какой лист загружается) и другие параметры
df = pd.read_csv('/Users/dmitrii/Documents/GitHub/DE-101/Module01/Sample - Superstore.csv', delimiter=';')

In [3]:
# смотрим первые значения, чтобы убедиться, что импорт выполнен успешно
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Manager,Returned
0,1,CA-2018-152156,08.11.2018,11.11.2018,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,"261,96",2,0,"41,9136",Cassandra Brandow,No
1,2,CA-2018-152156,08.11.2018,11.11.2018,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...","731,94",3,0,"219,582",Cassandra Brandow,No
2,3,CA-2018-138688,12.06.2018,16.06.2018,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,"14,62",2,0,"6,8714",Anna Andreadi,No
3,4,US-2017-108966,11.10.2017,18.10.2017,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,"957,5775",5,"0,45","-383,031",Cassandra Brandow,No
4,5,US-2017-108966,11.10.2017,18.10.2017,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,"22,368",2,"0,2","2,5164",Cassandra Brandow,No


In [4]:
# посмотрим на тип данных в которые Pandas автоматически преобразовал наши данные
# видим, что можно улучшить - Order Date и Ship Date преобразовать в datetime
# а Sales, Profit и Discount в float
df.dtypes

Row ID             int64
Order ID          object
Order Date        object
Ship Date         object
Ship Mode         object
Customer ID       object
Customer Name     object
Segment           object
Country           object
City              object
State             object
Postal Code      float64
Region            object
Product ID        object
Category          object
Sub-Category      object
Product Name      object
Sales             object
Quantity           int64
Discount          object
Profit            object
Manager           object
Returned          object
dtype: object

In [5]:
# превращаем object в datetime
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])

In [6]:
# object в int
df['Sales'] = df['Sales'].astype(str)
df['Sales'] = [i.replace(',','.') for i in df['Sales']]
df['Sales'] = pd.to_numeric(df['Sales'], errors='coerce')

In [7]:
df['Discount'] = df['Discount'].astype(str)
df['Discount'] = [i.replace(',','.') for i in df['Discount']]
df['Discount'] = pd.to_numeric(df['Discount'], errors='coerce')

In [8]:
df['Profit'] = df['Profit'].astype(str)
df['Profit'] = [i.replace(',','.') for i in df['Profit']]
df['Profit'] = pd.to_numeric(df['Profit'], errors='coerce')

In [9]:
# снова смотрим наши типы данных
df.dtypes

Row ID                    int64
Order ID                 object
Order Date       datetime64[ns]
Ship Date        datetime64[ns]
Ship Mode                object
Customer ID              object
Customer Name            object
Segment                  object
Country                  object
City                     object
State                    object
Postal Code             float64
Region                   object
Product ID               object
Category                 object
Sub-Category             object
Product Name             object
Sales                   float64
Quantity                  int64
Discount                float64
Profit                  float64
Manager                  object
Returned                 object
dtype: object

In [10]:
# теперь импортируем необходимые библиотеки для отправки данных в SQL
import psycopg2
from sqlalchemy import create_engine

In [11]:
# создаем подключение
engine = create_engine('postgresql+psycopg2://dmitrii:@127.0.0.1/dmitrii')

In [ ]:
# отправляем данные и смотрим все ли прошло без ошибок
df.to_sql('superstore', con=engine, if_exists='replace')